In [2]:
import re
import spacy
import os
import shutil
from pathlib import Path

In [3]:
"""
Ważna uwaga:
Pliki z lekturami (trzeba funkcji ustawić argument is_fragment=False) do wczytania muszą mieć następującą strukturę:
Autor

Tytuł(dowolna ilość linii)

Ewentualny numer ISBN

Tekst(dowolna ilość linii)
Ewentualna stopka po treście(przypis od wolnychlektur): -----
"""
"""
Przy podawaniu fragmentu(is_fragment=True) skanowany jest cały plik, więc struktura tekstu jest dowolna
"""
"""
tworzy worek słów z pliku
file_name - nazwa pliku z której utworzyc worek słów; with_stop_words - True/False mówiący czy usuwać stop wordsy czy nie
is_fragment - True/False - mówi czy podawany plik to fragment do sprawdzenia, czy cały utwór
"""
def create_bag_of_words_from_file(file_name, is_fragment, with_stop_words=False, lemma=True, lemma_accuracy=False):
#struktura worka: indeks to indeks, elementem jest lista ze słowem i liczebnością
    bag = {}
    stop_words = []
#tablica znaków interpunkcyjnych do pozbycia się z wyrazów
#TODO: jakieś jeszcze inne znaki?
    punc = '''…!”„()-[]»{};:'"\,<>./?@#$%^&*_~«—'''+ "\n" + " "
#ewentualne przypisanie stopwordsów
    if not with_stop_words:
        stop_words = get_stop_words_from_file('stopWords.txt')
        
    with open(file_name,'r', encoding='utf-8') as file:
#autor
        if not is_fragment:
            bag[0] = get_author(file)
            
        for line in file:
            for word in line.split():
#jak dotrze do stopki-kończ
                if not is_fragment and word == "-----":
                    return bag
                
            words = words_from_line(line, lemma, lemma_accuracy, punc)
            for word in words:
                word = word.lower()
#jeśli jest stop wordsem- nie uwzględniamy (jeśli w funkcji było True, to 'stop_words' jest puste, czyli nie wejdzie do ifa)    
                if word in stop_words:
                    continue
#dodawanie nowego słowa do worka, jeśli jeszcze go nie ma
                if word not in bag and (len(word) != 0):
                    bag[word] = 1
#inkrementacja liczności słowa, jeśli już istnieje w worku
                elif len(word) != 0:
                    bag[word] += 1
    return bag

"""
Tworzy tablicę słów w zależności od tego, czy używamy lematyzacji
- Jeżeli nie używamy lematyzacji to usuwa ze słów ewentualne znaki interpunkcyjne
- Jeżeli używamy lematyzacji to usuwa nadmierne spacje, generuje tokeny i zamienia je na słowa
w wersji podstawowej zależnie od kontekstu w linijce. Pomija słowa będące znakami interpunkcyjnymi
"""
def words_from_line(line, lemma, lemma_accuracy, punc):
    words = []
    if not lemma:
        for word in line.split():
            for ele in word:
                if ele in punc:
                    word = word.replace(ele, "")
            words.append(word)
    
    else:
        line = re.sub(' +', ' ', line)
        package = "pl_core_news_"
        if lemma_accuracy:
            package += "sm"
        else:
            package += "lg"
        nlp = spacy.load(package)
        tokens = nlp(line, disable=["parser", "ner"])
        for token in tokens:
            word = token.lemma_
            if word in punc:
                    continue
            words.append(word)
        
    return words

"""
word_bags - lista stworzonych z lektur worków słów
WAŻNE: w każdym worku pod indeksem 0(liczba, nie string) musi znajdować się autor danego utworu
WAŻNE2: używać przed użyciem funkcji count_percents na workach
struktura outputu: słownik, gdzie kluczem są autorzy, a wartościami worek słów(z których zostanie wyrzucony klucz 0-z autorem)
"""
def combine_word_bags(word_bags):
    combined = {}
#przechodzenie po wszystkich workach z listy
    for bag in word_bags:
        keys = combined.keys()
#pobieranie autora z aktualnego worka i usuwanie go ze słownika
        author = bag[0]
        del bag[0]
#jeśli autor już jest w wynikowym worku, to aktualizujemy wystąpienia słów u niego
        if author in keys:
            for elem in bag:
#jeśli autor ma już dane słowo, to dodajemy ilość wystąpień z aktualnie iterowanego worka
                if elem in combined[author]:
                    combined[author][elem] += bag[elem]
#jeśli słowa jeszcze nie ma, to dodajemy rekord
                else:
                    combined[author][elem] = bag[elem]
#jeśli autora nie ma jeszcze w wynikowym worku, to dodajemy rekord, przypisując do niego aktualny worek
        else:
            combined[author] = bag
    return combined

"""
funkcja zmienia ilosć wystąpień danego słowa na procent wartość wystąpień w tekście
"""
def count_percents(word_bag):
    amount = 0
#zliczanie ilości słów w worku
    for elem in word_bag:
        if elem == 0:
            continue
        amount += word_bag[elem]
#przeliczanie wystąpień na wartości procentowe
    for elem in word_bag:
        if elem == 0:
            continue
        word_bag[elem] = word_bag[elem]/amount*100
    return word_bag

"""
Funkcja zapisuje worki słów do plików. Folderem jest BagsOfWords/, gdzie pliki tekstowe mają tytuł autor_worka.txt.
Format pojedyńczej linii z rekordem: słowo ilość_wystąpień
word_bags- słownik worków, gdzie kluczem jest autor, a wartością worek
"""
def save_word_bags_to_file(word_bags):
    if os.path.isdir('BagOfWords'):
        shutil.rmtree('BagOfWords')
    os.mkdir('BagOfWords')
    for key in word_bags.keys():
        bag = word_bags[key]
        file = open("BagOfWords/" + key + ".txt", "w", encoding="utf-8")
        for b_keys in bag.keys():
            file.write(b_keys + " " + str(bag[b_keys]) + '\n')
        file.close()
      
"""
Funkcja pobiera już wcześniej utworzone worki słów z folderu
directory- folder z workami słów 
WAŻNE:
directory podawać bez ukośnika! Czysta nazwa!
pliki z workami słów danych autorów muszą mieć za nazwę imię i nazwisko tego autora
"""
def get_word_bags_from_directory(directory):
    files = os.listdir(directory)
    bags = {}
    for author in files:
        tmp_bag = {}
        file = open(directory + "\\" + author, encoding='utf-8')
        for line in file:
            line = line.split()
            tmp_bag[line[0]] = line[1]
        bags[repr(author).replace(".txt", "").replace("'", "")] = tmp_bag
    return bags
    
            

In [4]:
#To są różne funkcje pomocnicze
"""
funkcja pobiera stop wordsy z pliku i zwraca je jako listę
"""
def get_stop_words_from_file(file_name):
    stops = []
    borders = ["\\n", "'"]
    with open(file_name, encoding='utf-8') as f:
        for line in f:
            single_line = repr(line)
            for char in borders:
                single_line = single_line.replace(char,"")
            if not(single_line == ""):
                stops.append(single_line)
    return stops

"""
funkcja pobiera autora utworu, przeskakuje tytuł i przeskakuje number ISBN
"""
def get_author(file):
    author = file.readline()
    author = author.replace("\n","")
    author = list(author)
    if author[0] == " ":
        author[0] = ""
    author = "".join(author)
    
    file.readline()
    while not (file.readline()  == "\n"):
        continue
    position_before = file.tell()
    if not file.readline().startswith('ISBN'):
        file.seek(position_before)
    return author

In [15]:
"""
Metoda pomocnicza - automatycznie tworzy worki słów (od parametru combined zależy czy będą to 
worki słów każdej książki, czy wspólne dla każdego autora)
"""
def create_bags_of_words_from_authors(combined=True, with_stop_words=False, lemma=True, lemma_accuracy=False):
    books_list = os.listdir('Books')
    word_bags = []
    for book in books_list:
        word_bags.append(create_bag_of_words_from_file('Books/'+book, with_stop_words, lemma, lemma_accuracy))            
    if combined:
        word_bags = combine_word_bags(word_bags)            
    return word_bags

In [5]:
#to jest do moich testów, ale tak to w sumie będzie juzpotem wczytywane
books_list = os.listdir('Books')
word_bags = []
for book in books_list:
    word_bags.append(create_bag_of_words_from_file('Books/'+book, False, False, False))
word_bags = combine_word_bags(word_bags)
print(len(word_bags))
for k in word_bags.keys():
    print(k)
for k in word_bags.keys():
    word_bags[k] = count_percents(word_bags[k])
save_word_bags_to_file(word_bags)
#########################################
#Tutaj sytuacja, gdy uruchamiamy po raz kolejny i już mamy gotowe, zapisane worki słów
dicti = get_word_bags_from_directory("BagOfWords")

11
Bolesław Prus
Sofokles
Krzysztof Kamil Baczyński
Adam Mickiewicz
Juliusz Słowacki
Tadeusz Borowski
Stefan Żeromski
Jan Kochanowski
Aleksander Fredro
Henryk Sienkiewicz
Maria Konopnicka


In [6]:
print(dicti['Jan Kochanowski'])

{'panie': '0.11029411764705882', 'praca': '0.03676470588235294', 'zdarzenie': '0.03676470588235294', 'raczyż': '0.03676470588235294', 'błogosławieństwo': '0.03676470588235294', 'dać': '0.11029411764705882', 'końca': '0.14705882352941177', 'inszy': '0.07352941176470588', 'niechaj': '0.22058823529411764', 'pałace': '0.03676470588235294', 'marmurowe': '0.03676470588235294', 'szczerym': '0.03676470588235294', 'złotogłowem': '0.03676470588235294', 'ściany': '0.03676470588235294', 'obijają': '0.03676470588235294', 'mieszkam': '0.03676470588235294', 'gniaździe': '0.03676470588235294', 'ojczystym': '0.03676470588235294', 'mię': '0.4044117647058824', 'zdrowiem': '0.03676470588235294', 'opatrz': '0.07352941176470588', 'sumnieniem': '0.03676470588235294', 'czystym': '0.07352941176470588', 'pożywieniem': '0.03676470588235294', 'ućciwym': '0.07352941176470588', 'ludzką': '0.07352941176470588', 'życzliwością': '0.03676470588235294', 'obyczajmi': '0.03676470588235294', 'znośnymi': '0.0367647058823529